## Simulate Price by BS (no Arbitrage)

In [1]:
import numpy as np
import pandas as pd
from enum import Enum
from scipy.stats import norm
import pandas_market_calendars as mcal

In [2]:
class PayoffType(str, Enum):
    Call = 'Call'
    Put = 'Put'

def computeR(S, C, P, T, K):
    r = -1/T * np.log((S-C+P)/K)
    return r

amzn_t = mcal.get_calendar('NASDAQ')        
def computeExpiry(t0, T):
    trading_days = amzn_t.valid_days(start_date=t0, end_date=T)
    expiry = len(trading_days) / 252
    return expiry


def bsPrice(S, r, vol, T, K, payoffType):
    fwd = S * np.exp(r * T)
    stdev = vol * np.sqrt(T)
    d1 = np.log(fwd / K) / stdev + stdev / 2
    d2 = d1 - stdev
    
    if payoffType == PayoffType.Call:
        return np.exp(-r * T) * (fwd * norm.cdf(d1) - K * norm.cdf(d2))
    elif payoffType == PayoffType.Put:
        return np.exp(-r * T) * (K * norm.cdf(-d2) - fwd * norm.cdf(-d1))
    else:
        raise Exception("not supported payoff type", payoffType)

    

**Use same t0, T1, T2 as real, choose different K as strike**

In [3]:
S = pd.read_csv('adjusted_stocks.csv')
S_amzn = S.iloc[:,0:2]
S_amzn['Date'] = pd.to_datetime(S_amzn['Date'])
T1 = '2018-01-19'
T2 = '2018-04-20'
t0List = S_amzn[S_amzn['Date'] <= T1]
t0List = list(t0List['Date'].unique())
strikeList = list(range(800, 1751, 5))
r = 0.02
vol = 0.20

**Simulate Call Option Price**

In [4]:
# Compute the option prices for T1
results_T1 = []
for t0 in t0List:
    for K in strikeList:
        expiry = computeExpiry(t0, T1)
        amzn_price = S_amzn.loc[S_amzn['Date'] == t0, 'AMZN'].iloc[0]
        price = bsPrice(amzn_price, r, vol, expiry, K, PayoffType.Call)
        results_T1.append({'t0': t0, 'T1': T1, 'K1': K, 'C1': price})

# Compute the option prices for T2
results_T2 = []
for t0 in t0List:
    for K in strikeList:
        expiry = computeExpiry(t0, T2)
        amzn_price = S_amzn.loc[S_amzn['Date'] == t0, 'AMZN'].iloc[0]
        price = bsPrice(amzn_price, r, vol, expiry, K, PayoffType.Call)
        results_T2.append({'t0': t0, 'T2': T2, 'K2': K, 'C2': price})

C1 = pd.DataFrame(results_T1)
C2 = pd.DataFrame(results_T2)

**Same Strike Price as Pairs**

In [5]:
result = pd.merge(C1, C2, left_on=['t0', 'K1'], right_on=['t0', 'K2'])
result['S0'] = result['t0'].apply(lambda x: S_amzn.loc[S_amzn['Date'] == x, 'AMZN'].values[0]) 
result

,t0,T1,K1,C1,T2,K2,C2,S0
0,2018-01-02,2018-01-19,800,3.898350e+02,2018-04-20,800,393.824109,1189.010010
1,2018-01-02,2018-01-19,805,3.848401e+02,2018-04-20,805,388.855034,1189.010010
2,2018-01-02,2018-01-19,810,3.798453e+02,2018-04-20,810,383.886164,1189.010010
3,2018-01-02,2018-01-19,815,3.748504e+02,2018-04-20,815,378.917544,1189.010010
4,2018-01-02,2018-01-19,820,3.698556e+02,2018-04-20,820,373.949223,1189.010010
...,...,...,...,...,...,...,...,...
2478,2018-01-19,2018-01-19,1730,1.634671e-117,2018-04-20,1730,0.104716,1294.579926
2479,2018-01-19,2018-01-19,1735,8.041292e-120,2018-04-20,1735,0.095159,1294.579926
2480,2018-01-19,2018-01-19,1740,3.812901e-122,2018-04-20,1740,0.086437,1294.579926
2481,2018-01-19,2018-01-19,1745,1.743309e-124,2018-04-20,1745,0.078480,1294.579926


**No pair Result**

In [6]:
shuffled_C2 = C2.sample(frac=1).reset_index(drop=True)
result_noPairStrike = pd.merge(C1.reset_index(drop=True), shuffled_C2[['T2', 'K2', 'C2']], left_index=True, right_index=True)
result_noPairStrike['t0'] = C1['t0']
result_noPairStrike['S0'] = result_noPairStrike['t0'].apply(lambda x: S_amzn.loc[S_amzn['Date'] == x, 'AMZN'].values[0])
result_noPairStrike


,t0,T1,K1,C1,T2,K2,C2,S0
0,2018-01-02,2018-01-19,800,3.898350e+02,2018-04-20,1245,48.723377,1189.010010
1,2018-01-02,2018-01-19,805,3.848401e+02,2018-04-20,965,345.445281,1189.010010
2,2018-01-02,2018-01-19,810,3.798453e+02,2018-04-20,1510,5.718500,1189.010010
3,2018-01-02,2018-01-19,815,3.748504e+02,2018-04-20,1055,207.413698,1189.010010
4,2018-01-02,2018-01-19,820,3.698556e+02,2018-04-20,1320,17.840036,1189.010010
...,...,...,...,...,...,...,...,...
2478,2018-01-19,2018-01-19,1730,1.634671e-117,2018-04-20,1430,12.828318,1294.579926
2479,2018-01-19,2018-01-19,1735,8.041292e-120,2018-04-20,1570,1.031274,1294.579926
2480,2018-01-19,2018-01-19,1740,3.812901e-122,2018-04-20,1110,203.994370,1294.579926
2481,2018-01-19,2018-01-19,1745,1.743309e-124,2018-04-20,1245,60.894436,1294.579926


In [7]:
result.to_csv('Same_Strike_Price.csv', header=True)
result_noPairStrike.to_csv('NoPair_Strike_Price.csv', header=True)